In [ ]:
import os
from typing import NoReturn, List, Optional
import psycopg2
from psycopg2.extensions import connection, cursor
from langchain_core.tools import tool
from dotenv import load_dotenv

load_dotenv()


In [ ]:

class PeqiiDatabaseManager:

    DATABASE_URL: str = os.getenv("DATABASE_URL")

    def __init__(self) -> None:
        self.conn: connection = psycopg2.connect(self.DATABASE_URL)
        self.cur: cursor = self.conn.cursor()

    def _close_connection(self) -> NoReturn:
        """
        Fecha a conexão com o banco de dados.
        """
        self.cur.close()
        self.conn.close()

    # @tool
    def execute_query(self, query_params: dict) -> Optional[List[tuple]]:
        """
        Executa uma consulta SQL no banco de dados.

        :param query_params: Dicionário onde a primeira chave-valor representa o nome da tabela e os demais representam os nomes das colunas e valores.
        :return: Lista de tuplas contendo os resultados da consulta ou None em caso de falha.
        """
        try:
            table_name = list(query_params.keys())[0]
            columns = list(query_params.keys())[1:]
            values = [query_params[col] for col in columns]

            query = f"SELECT * FROM {table_name} WHERE " + " AND ".join([f"{col} = %s" for col in columns])
            self.cur.execute(query, values)
            results = self.cur.fetchall()
            return results
        except Exception as e:
            print(f"Erro ao executar a consulta: {e}")
            return None
        finally:
            self._close_connection()


In [ ]:
query_params = {
    "table_name": "contacts"
}

db_manager = PeqiiDatabaseManager()
results = db_manager.execute_query(query_params)
print(results)

# Teste Toolkit LangChain

In [1]:
import sqlite3
import requests
from dotenv import load_dotenv
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool


def get_engine_for_chinook_db():
    """Pull sql file, populate in-memory database, and create engine."""
    url = "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql"
    response = requests.get(url)
    sql_script = response.text

    connection = sqlite3.connect(":memory:", check_same_thread=False)
    connection.executescript(sql_script)
    return create_engine(
        "sqlite://",
        creator=lambda: connection,
        poolclass=StaticPool,
        connect_args={"check_same_thread": False},
    )


engine = get_engine_for_chinook_db()

db = SQLDatabase(engine)

In [2]:
load_dotenv()

from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(model="gpt-4o")

In [3]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [4]:
toolkit.get_tools()



[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10ead7290>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10ead7290>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10ead7290>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

In [5]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

/Users/morossini/anaconda3/envs/hub/lib/python3.11/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [6]:
from pprint import pprint
pprint(prompt_template)


ChatPromptTemplate(input_variables=['dialect', 'top_k'], metadata={'lc_hub_owner': 'langchain-ai', 'lc_hub_repo': 'sql-agent-system-prompt', 'lc_hub_commit_hash': '31156d5fe3945188ee172151b086712d22b8c70f8f1c0505f5457594424ed352'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['dialect', 'top_k'], template='You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information r

In [7]:
dir(prompt_template)

['Config',
 'InputType',
 'OutputType',
 '__abstractmethods__',
 '__add__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__orig_bases__',
 '__parameters__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__ror__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__sub

In [8]:
print(prompt_template.messages[0].prompt.template.replace('\\n', '\n'))

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see 

In [9]:
assert len(prompt_template.messages) == 1
print(prompt_template.input_variables)

['dialect', 'top_k']


In [10]:
system_message = prompt_template.format(dialect="SQLite", top_k=5)
print(system_message)

System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see w

In [11]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(
    llm, toolkit.get_tools(), state_modifier=system_message
)

In [12]:
example_query = "Which country's customers spent the most?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Which country's customers spent the most?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_Y7Yryf2wff3H9liUKchiRaHy)
 Call ID: call_Y7Yryf2wff3H9liUKchiRaHy
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_AkZ5GRONn3Vw5y6SjM3qqMKp)
 Call ID: call_AkZ5GRONn3Vw5y6SjM3qqMKp
  Args:
    table_names: Customer, Invoice
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company"

In [15]:
example_query = "What are the available tables?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What are the available tables?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_pPQDp0nYo8dk9oG2PAjugsVO)
 Call ID: call_pPQDp0nYo8dk9oG2PAjugsVO
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================

The available tables in the database are:

1. Album
2. Artist
3. Customer
4. Employee
5. Genre
6. Invoice
7. InvoiceLine
8. MediaType
9. Playlist
10. PlaylistTrack
11. Track

Feel free to ask about any specific information you need from these tables!


In [16]:
example_query = "Who are the 3 top selling artists?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Who are the 3 top selling artists?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_FrNgCt0jbhKPuq5HsnXGuj46)
 Call ID: call_FrNgCt0jbhKPuq5HsnXGuj46
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_ELpLJ5NMJjIo9TtPnrtaQCEu)
 Call ID: call_ELpLJ5NMJjIo9TtPnrtaQCEu
  Args:
    table_names: Artist, InvoiceLine, Track, Album
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("

In [17]:
example_query = "What are the 3 most popular genres?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What are the 3 most popular genres?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_3kGbTfRr4qB8dfrMkuU3XjIC)
 Call ID: call_3kGbTfRr4qB8dfrMkuU3XjIC
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_L0mJi2NYD42kvIPAFNCGyh9w)
 Call ID: call_L0mJi2NYD42kvIPAFNCGyh9w
  Args:
    table_names: Genre,Track
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Genre" (
	"GenreId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("GenreId")
)

/*
3 rows from Genre table:
GenreId	Name
1	Rock
2

In [18]:
example_query = "What are the 10 most popular albums?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What are the 10 most popular albums?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_15FFX2ZhIs1tCwLZyJZmaPN7)
 Call ID: call_15FFX2ZhIs1tCwLZyJZmaPN7
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_EWgBPNdobbTcXDPfzuFeOdyA)
 Call ID: call_EWgBPNdobbTcXDPfzuFeOdyA
  Args:
    table_names: Album, InvoiceLine, Invoice
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("Albu

In [19]:
example_query = "What are the 3 least popular songs?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What are the 3 least popular songs?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_NUICqW5jTy2koOC9MPmZAE6d)
 Call ID: call_NUICqW5jTy2koOC9MPmZAE6d
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_M9hupSXNo0T3sx5uig6cifL6)
 Call ID: call_M9hupSXNo0T3sx5uig6cifL6
  Args:
    table_names: Track, InvoiceLine
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "InvoiceLine" (
	"InvoiceLineId" INTEGER NOT NULL, 
	"InvoiceId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	"UnitPrice" NUMERIC

In [20]:
example_query = "Are there any Brazilian customers?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Are there any Brazilian customers?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_EWgBPNdobbTcXDPfzuFeOdyA)
 Call ID: call_EWgBPNdobbTcXDPfzuFeOdyA
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_15FFX2ZhIs1tCwLZyJZmaPN7)
 Call ID: call_15FFX2ZhIs1tCwLZyJZmaPN7
  Args:
    table_names: Customer
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 


In [21]:
example_query = "Are there any customers from Yemen?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Are there any customers from Yemen?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_Lknex7JvISDQH00KaucCvxFI)
 Call ID: call_Lknex7JvISDQH00KaucCvxFI
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_F2hC5IYtEfAsNUcJGkg5j0vg)
 Call ID: call_F2hC5IYtEfAsNUcJGkg5j0vg
  Args:
    table_names: Customer
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 

In [22]:
example_query = "Who are the top 3 and the bottom 3 media types?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Who are the top 3 and the bottom 3 media types?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_JSlXLEtzQBp5UROS6pANlLC6)
 Call ID: call_JSlXLEtzQBp5UROS6pANlLC6
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_jZQD4a57ljKep2b3hBumdRzr)
 Call ID: call_jZQD4a57ljKep2b3hBumdRzr
  Args:
    table_names: MediaType, Track
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "MediaType" (
	"MediaTypeId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("MediaTypeId")
)

/*
3 rows from Media